In [21]:
import os
os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

In [22]:
# # Get test_q_keys
# import random
# import pandas as pd
# from src.utils import get_file_from_s3

# survey_name = "American_Trends_Panel_W34"
# random.seed(42)
# survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
# q_keys = list(survey_df['key'])
# test_q_keys = random.choices(q_keys, k=5)

# print(test_q_keys)

In [23]:
import random
random.seed(42)

from src.utils import TEST_KEY_MAPPING

test_q_keys_mapping = TEST_KEY_MAPPING

# test_q_keys_mapping = {
#     "American_Trends_Panel_W26": ['GUNRESPNOKIDSB_W26', 'WORLDDANGER_W26', 'GUNIDENTITY_W26', 'REASONGUNC_W26', 'GUNRESPKIDSC_W26'],
#     "American_Trends_Panel_W27": ['CAREGIV3A_W27', 'PREDICTA_W27', 'WORK3C_W27', 'CARS10D_W27', 'CAREGIV7_W27'],
#     "American_Trends_Panel_W29": ['TRAITSD_W29', 'NOWSMK_NHIS_W29', 'HELPHURTE_W29', 'HELPHURTA_W29', 'GIRLSF2A_W29'],
#     "American_Trends_Panel_W32": ['NEIGHINTERB_W32', 'SATLIFEC_W32', 'SUCCESSIMPA_W32', 'COMMIMPE_W32', 'GROWUPUSR_W32'],
#     "American_Trends_Panel_W34": ['MED4A_W34', 'SCI2A_W34', 'FUD22_W34', 'EAT3H_W34', 'MED6D_W34']
# }


# Hyper parameters

In [24]:

survey_names = [
    # "American_Trends_Panel_W26",
    # "American_Trends_Panel_W27",
    # "American_Trends_Panel_W29",
    # "American_Trends_Panel_W32",
    "American_Trends_Panel_W34"
]

def skip_exp(filename):
    # # check sonnet vs haiku pred
    # if 'v11' in filename: return True
    # if ('0830' in filename and 'prompt2' in filename): return False
    # if ('sonnetpred' in filename): return False
    # return True

    # # check prompts
    # if 'haikupred' not in filename: return True
    # # if 'v12' in filename and 'prompt2' in filename: return False
    # if 'v11' in filename: 
    #     if 'prompt2' in filename: return False
    #     if 'prompt3' in filename: return False
    #     if 'prompt4' in filename: return False
    # if 'v10' in filename and 'prompt2' in filename: return False
    # return True

    # # check runs
    # if 'haikupred' not in filename: return True
    # if '0831' in filename and 'prompt3' in filename: return False
    # if 'v11' in filename:
    #     if 'prompt3' in filename: return False
    # if 'v10' in filename and 'prompt2' in filename: return False
    # return True

    # check known test
    if 'v17' in filename and '0904' in filename: return True
    if 'v17' in filename: return False
    if 'v13' not in filename and 'v16' not in filename: return True
    if '0901' not in filename and '0902' not in filename: return True
    if 'history_demo' in filename and '0901' in filename: return True

    # if '0831_v13' not in filename: return True

def get_exp(filename):
    if filename[-5:] == '.json': filename = filename[:-5]
    
    # segs = ['history', 'demo', 'persona', 'v11', 'v10', 'prompt3', 'prompt4', 'run1', 'run2', 'run3']
    segs = ['history', 'demo', 'persona', 'withname', 'all' 'allq']
    for level in ['13', '16', '17']: segs.append('v'+level)
    for train in ['200', '500', '2000', 'All']: segs.append('train'+train)
    for top in range(10, 110, 10): segs.append('top'+str(top))
    for skew in ['5', '3', '10']: segs.append('skew'+skew)
    for usedemo in ['True', 'False']: segs.append('usedemo'+usedemo)
    for run in ['run1', 'run2', 'run3']: segs.append(run)
    for setting in ['queryponly', 'querydp', 'querydonly']: segs.append(setting)

    exp_elements = [_ for _ in filename.split('_') if _ in segs]
    final = '_'.join(exp_elements)
    return final

# Main

In [25]:
import json
import os
import pandas as pd
import numpy as np
import pandas as pd
import random

from collections import Counter
from src.utils import get_file_from_s3

def get_full_results(survey_name):
    res = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue
        exp = get_exp(filename)
        # exp = filename
        if exp not in res:
            res[exp] = []
            
        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", sum([_['is_correct'] for _ in data])/len(data)]

    for key in res:
        print(key, len(res[key]))

    df = pd.DataFrame(res)
    df = df[sorted(df.columns)]
    df = df.transpose()

    acc_df = df
    acc_df = acc_df.rename(columns={0: "acc_cnt", 1: "acc"})
    # acc_df.sort_values(by='acc', ascending=False)


    test_q_keys = test_q_keys_mapping[survey_name]
    print(test_q_keys)
    survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
    sub_df = survey_df[survey_df['key'].isin(test_q_keys)]

    wd_question_mappings = {row['key']: dict(zip(eval(row['references'].lower()), eval(row['option_ordinal']))) for _, row in sub_df.iterrows()}
    wd_question_mappings

    records = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue

        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        
        exp = get_exp(filename)

        res = []
        pred_missing_cnt = 0
        gold_missing_cnt = 0
        for entry in data:
            q_idx = entry['q_idx']
            q_key = test_q_keys[q_idx]
            pos_mapping = wd_question_mappings[q_key]
            prediction = entry['prediction'].lower()
            gold_answer = entry['gold_answer'].lower()
            if gold_answer not in pos_mapping:
                gold_missing_cnt += 1
                continue
            if prediction not in pos_mapping:
                pred_missing_cnt += 1
                if prediction not in entry['references']:
                    refs = '/'.join(entry['references'].split('\n'))
                    print(f"Prediction: |{prediction}|, Gold: |{gold_answer}|, References: |{refs}|")
                continue
                prediction = random.choice(list(pos_mapping.keys()))
            pred_pos = pos_mapping[prediction]
            gold_pos = pos_mapping[gold_answer]
            res.append(abs(pred_pos-gold_pos))
        records[exp] = [Counter(res), np.mean(res), pred_missing_cnt, gold_missing_cnt]

    q_wd_df = pd.DataFrame(records)
    q_wd_df = q_wd_df[sorted(q_wd_df.columns)].transpose()
    q_wd_df = q_wd_df.rename(columns={0: "q_1wd_cnt", 1: "q_1wd", 2: "q_1wd_pmc", 3: "q_1wd_gmc"})
    q_wd_df

    merged_df = pd.merge(acc_df, q_wd_df, left_index=True, right_index=True)
    # merged_df = pd.merge(merged_df, s_wd_df, left_index=True, right_index=True)
    # merged_df = merged_df[['wd_avg', 'wd_std', 'acc_avg', 'acc_std']]

    merged_df = merged_df.sort_values(by='q_1wd', ascending=True)
    merged_df = merged_df.sort_values(by='acc', ascending=False)

    # merged_df.sort_values(by='acc_avg', ascending=False)
    # merged_df.sort_values(by='wd_avg', ascending=True)
    # merged_df = merged_df.sort_values(by='s_1wd', ascending=True)

    merged_df['acc_cnt'] = merged_df['acc_cnt'].apply(lambda x: '\''+str(x))
    merged_df

    return merged_df

# Results

In [26]:
print(merged.to_csv())

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v13_history_demo_run3,'=913/1518,0.6014492753623188,"Counter({0.0: 913, 1.0: 476, 2.0: 104, 3.0: 17})",0.4867549668874172,0,8
v13_history_demo_run2,'=907/1518,0.5974967061923584,"Counter({0.0: 907, 1.0: 483, 2.0: 104, 3.0: 16})",0.48940397350993375,0,8
v13_history_demo_run1,'=905/1518,0.5961791831357048,"Counter({0.0: 905, 1.0: 481, 2.0: 108, 3.0: 16})",0.49337748344370863,0,8
v16_queryponly_trainAll_top100_skew3_withname_history_demo_run1,'=903/1518,0.5948616600790514,"Counter({0.0: 903, 1.0: 497, 2.0: 98, 3.0: 12})",0.4827814569536424,0,8
v17_queryponly_trainAll_top100_skew3_withname_history_demo_run3,'=903/1518,0.5948616600790514,"Counter({0.0: 903, 1.0: 493, 2.0: 103, 3.0: 11})",0.4847682119205298,0,8
v17_queryponly_trainAll_top100_skew3_withname_history_demo_run2,'=899/1518,0.5922266139657444,"Counter({0.0: 899, 1.0: 496, 2.0: 103, 3.0: 12})",0.48874172185430464,0,8
v16_queryponly_trainAll_top100_skew3_withname_history_demo_run2,'=8